# Primer - BBC

* Uporabimo bazo [`bbc.db`](bbc.db).
* Izpišimo imena vseh evropskih držav.

Uvozimo podatkovni vmesnik in se povežimo na bazo.

In [ ]:
import sqlite3 as dbapi
povezava = dbapi.connect("bbc.db")
kazalec = povezava.cursor()

Zapišimo naš ukaz in ga izvedimo.

In [ ]:
sql = "SELECT * FROM bbc WHERE region = 'Europe'"
kazalec.execute(sql)        # izvedemo ukaz

Preberimo celoten rezultat v seznam in izpišimo imena držav.

In [ ]:
zapisi = kazalec.fetchall() # preberemo tabelo z rezultati
for vrstica in zapisi:
    print(vrstica[0])      # ime je prvi element nabora

In [ ]:
zapisi

Namesto tega bi lahko z zanko `for` brali zapise enega po enega.

In [ ]:
for ime, regija, povrsina, prebivalstvo, bdp in kazalec:
    print(ime, povrsina)

Lahko pa tudi ročno beremo vrstico po vrstico.

In [ ]:
kazalec.fetchone()

In [ ]:
next(kazalec)

Pristope lahko tudi kombiniramo.

In [ ]:
for ime, *_, bdp in kazalec:
    print(ime, bdp)
    if ime == 'Slovenia':
        break
print(kazalec.fetchmany(5))
print(kazalec.fetchall())

Nazadnje zaprimo kurzor in povezavo na bazo.

In [ ]:
kazalec.close()
povezava.close()

# Še en primer

In [ ]:
import sqlite3 as dbapi
# Povežemo se na novo bazo - s tem jo ustvarimo
conn = dbapi.connect("testdb.sqlite")
cur = conn.cursor()                        # Odpremo kazalec
cur.execute("DROP TABLE IF EXISTS test;")  # Zbrišemo tabelo, če že obstaja
# Izvedemo ukaz - ustvarimo tabelo
cur.execute("""
      CREATE TABLE test (
        id   integer PRIMARY KEY AUTOINCREMENT,
        num  integer,
        data text
      );
    """)
# Vstavimo podatke v tabelo
cur.execute("INSERT INTO test (num, data) VALUES (100, 'KU-KU');")
cur.execute("SELECT * FROM test;")         # Preberemo zapisane podatke
rezultat  = cur.fetchone()                 # Hočemo le eno vrstico
print(rezultat)                            # Izpiše se (1, 100, "KU-KU")
conn.commit()                              # Poskrbimo, da so spremembe trajne
# Zapremo povezave z bazo
cur.close()
conn.close()

Kaj pa, če gre kaj narobe? V vsakem primeru želimo zapreti povezave - to lahko dosežemo z blokom `finally`.

In [ ]:
import sqlite3 as dbapi
# Povežemo se na novo bazo - s tem jo ustvarimo
conn = dbapi.connect("testdb.sqlite")
try:
    cur = conn.cursor()                            # Odpremo kazalec
    try:
        cur.execute("DROP TABLE IF EXISTS test;")  # Zbrišemo tabelo, če že obstaja
        # Izvedemo ukaz - ustvarimo tabelo
        cur.execute("""
              CREATE TABLE test (
                id   integer PRIMARY KEY AUTOINCREMENT,
                num  integer,
                data text
              );
            """)
        # Vstavimo podatke v tabelo
        cur.execute("INSERT INTO test (num, data) VALUES (100, 'KU-KU');")
        cur.execute("SELECT * FROM test;")         # Preberemo zapisane podatke
        rezultat  = cur.fetchone()                 # Hočemo le eno vrstico
        print(rezultat)                            # Izpiše se (1, 100, "KU-KU")
        # Vstavimo še eno vrstico
        cur.execute("INSERT INTO test (id, num, data) VALUES (1, 200, 'kaj pa zdaj?');")
        # Tukaj pride do napake, saj bi se ponovila vrednost v stolpcu id
        conn.commit()                              # Poskrbimo, da so spremembe trajne
    finally:
        # Zapremo povezavo s kazalcem
        print("Zapiramo kazalec")
        cur.close()
except dbapi.IntegrityError as ex:
    conn.rollback()                                # Prekličemo spremembe
    print(f"Prišlo je do napake: {ex}!")
    cur = conn.cursor()                            # Odpremo nov kazalec, ker smo starega zaprli
    cur.execute("SELECT * FROM test;")             # Preberemo zapisane podatke
    rezultat  = cur.fetchone()                     # Hočemo le eno vrstico
    print(rezultat)                                # Ni take vrstice, dobimo None
    cur.close()
finally:
    # Zapremo povezavo z bazo
    print("Zapiramo povezavo z bazo")
    conn.close()

# Uporaba `with`

Z `ẁith` poskrbimo za potrditev transakcije ob uspešnem izvajanju oziroma za preklic ob napaki.

In [ ]:
import sqlite3 as dbapi
conn = dbapi.connect("testdb.sqlite")
cur = conn.cursor()
try:
    with conn:
        cur.execute("INSERT INTO test (num, data) VALUES (100, 'prva vrstica');")
        cur.execute("""
            INSERT INTO test (id, num, data)
            VALUES (2, 200, 'kaj pa zdaj?');
            """)
        cur.execute("INSERT INTO test (num, data) VALUES (300, 'še nekaj');")
except dbapi.IntegrityError as ex:
    print(f"Napaka: {ex}")

Poglejmo, kaj se nahaja v tabeli.

In [ ]:
cur.execute("SELECT * FROM test;")
cur.fetchall()

Kurzor in povezava sta ostala odprta, tako da ju na koncu zapremo.

In [ ]:
cur.close()
conn.close()

# Parametriziranje ukazov SQL

Parametre lahko vstavljamo v ukaze SQL tako, da na njihova mesta zapišemo `?` ali ime parametra za `:`, njihove vrednosti pa podamo z drugim parametrom metode `execute`.

In [ ]:
import sqlite3 as dbapi
conn = dbapi.connect("bbc.db")

In [ ]:
def drzava(conn, ime):
    cur = conn.cursor()
    try:
        cur.execute("SELECT * FROM bbc WHERE name = ?;", (ime, ))
        return cur.fetchone()
    finally:
        cur.close()

In [ ]:
drzava(conn, 'Slovenia')

In [ ]:
def prebivalstvo(conn, min, max):
    cur = conn.cursor()
    try:
        cur.execute("""
              SELECT * FROM bbc
               WHERE population BETWEEN ? AND ?;
            """, [min, max])
        return cur.fetchall()
    finally:
        cur.close()

In [ ]:
prebivalstvo(conn, 1000000, 10000000)

In [ ]:
def prebivalstvo_povrsina(conn, vrednost):
    cur = conn.cursor()
    try:
        cur.execute("""
              SELECT * FROM bbc
               WHERE population >= :vrednost AND
                     area >= :vrednost;
            """, {'vrednost': vrednost})
        return cur.fetchall()
    finally:
        cur.close()

In [ ]:
prebivalstvo_povrsina(conn, 2000000)

In [ ]:
conn.close()

# SQL injection

Če nismo pazljivi pri vstavljanju podatkov v stavke SQL, lahko zlonameren uporabnik doseže, da se izvedejo ukazi, ki jih razvijalec aplikacije ni predvidel.

Denimo, da imamo v bazi tabelo uporabnikov, ki hrani njihova uporabniška imena, gesla in zastavico, ali gre za administratorja.

**Opomba:** v praksi gesel nikoli ne hranimo v bazi v čisti obliki!

In [ ]:
import sqlite3 as dbapi
conn = dbapi.connect("testdb.sqlite")
conn.set_trace_callback(print)
cur = conn.cursor()

try:
    with conn:
        cur.executescript("""
              DROP TABLE IF EXISTS uporabnik;
              CREATE TABLE uporabnik (
                id              integer PRIMARY KEY AUTOINCREMENT,
                uporabnisko_ime text    NOT NULL UNIQUE,
                geslo           text    NOT NULL,
                admin           integer NOT NULL DEFAULT 0
              );
            """)
        cur.executemany("""
              INSERT INTO uporabnik (uporabnisko_ime, geslo, admin)
              VALUES (?, ?, ?)
            """, [('admin', '$kr1wn0G35l0', 1),
                  ('janez', 'geslo123', 0)])
finally:
    cur.close()

Denimo, da s sledečo funkcijo preverjamo podatke, ki jih uporabnik vnese v prijavni obrazec. Ob uspešni prijavi funkcija vrne par `(id, admin)`, kjer je `id` zaporedna številka uporabnika, `admin` pa zastavica, ki pove, ali je uporabnik administrator. Ob neuspešni prijavi (napačno uporabniško ime ali geslo) funkcija vrne `None`.

In [ ]:
def prijava_slaba(conn, uporabnisko_ime, geslo):
    cur = conn.cursor()
    try:
        cur.execute(f"""
              SELECT id, admin FROM uporabnik
               WHERE uporabnisko_ime = '{uporabnisko_ime}' AND
                     geslo = '{geslo}';
            """)
        return cur.fetchone()
    finally:
        cur.close()

Preverimo delovanje funkcije na nekaj primerih.

In [ ]:
prijava_slaba(conn, 'janez', 'geslo123')

In [ ]:
prijava_slaba(conn, 'janez', 'napacno_geslo')

In [ ]:
prijava_slaba(conn, 'micka', 'geslo123')

Ob uporabi zgornje funkcije lahko zlonameren uporabnik sestavi take vhodne podatke, da bo funkcija vrnila podatke uporabnika `admin` z administratorskimi pravicami, čeprav zanj ne pozna gesla.

In [ ]:
prijava_slaba(conn, "admin' OR 0 AND --", "")

Tudi, če uporabnik ni zlonameren, lahko morda povzroči napako v delovanju programa.

In [ ]:
prijava_slaba(conn, "franci", "ges'lce")

Zapišimo popravljeno funkcijo za prijavo.

In [ ]:
def prijava_dobra(conn, uporabnisko_ime, geslo):
    cur = conn.cursor()
    try:
        cur.execute("""
              SELECT id, admin FROM uporabnik
               WHERE uporabnisko_ime = ? AND
                     geslo = ?;
            """, (uporabnisko_ime, geslo))
        return cur.fetchone()
    finally:
        cur.close()

In [ ]:
prijava_dobra(conn, 'janez', 'geslo123')

In [ ]:
prijava_dobra(conn, 'janez', 'napacno_geslo')

In [ ]:
prijava_dobra(conn, 'micka', 'geslo123')

In [ ]:
prijava_dobra(conn, "admin' OR 0 AND --", "")

In [ ]:
prijava_dobra(conn, "franci", "ges'lce")

In [ ]:
conn.close()